In [13]:
from flask import Flask, render_template, request, url_for
from tensorflow.keras.models import load_model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model
from tensorflow.keras.utils import img_to_array
from tqdm import tqdm
import keras
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import datetime
import json
# from requests_toolbelt.multipart import decoder


In [2]:
# 이미지의 벡터화를 진행하는 코드
# inception v3 model 로드
model_v3 = InceptionV3(weights='imagenet')
# inception v3에서 마지막 레이어를 제거한 새로운 모델 생성
model_new = Model(model_v3.input, model_v3.layers[-2].output)
object_model=load_model('./model/object_model.h5')
scenery_model=load_model('./model/scenery_model.h5')
img_df = pd.read_csv("./csv/시연데이터최종본.csv")

In [3]:
flower = ['꽃',  '꽃스타그램']
food = ['먹스타그램', '맛스타그램', '먹방', '맛집', '디저트', '아침', '집밥', '먹스타', '맛있다', '냠냠', '야식',
         '맛있다그램', '먹부림', '요리', '마카롱', '간식', '부산맛집', '치킨', '온더테이블', '존맛', '푸드스타그램',
          '음식', '제주맛집', '점심', '저녁']
drink = ['술스타그램']
fashion = ['데일리룩', '오오티디', '패션', '옷스타그램', '패션스타그램', '뷰티', '스타일', '모델', '코디', '명품',
          '뷰티스타그램', '아웃핏', '데일리코디', '등원룩', '원피스', '옷', '메이크업'] 
nailart = ['네일아트', '젤네일', '네일', '네일스타그램']
love = ['럽스타그램', '데이트', '커플', '사랑', '내사랑']
baby = ['육아', '육아스타그램', '육아소통', '육아맘', '맘스타그램', '아들스타그램', '애스타그램', '도치맘', '인스타베이비',
         '아들맘', '아들', '딸바보', '가족', '딸', '육아일기', '딸맘', '베이비스타그램', '아기', '육아일상', '아기스타그램',
          '부부스타그램', '줌마그램', '아들바보', '베이비', '베이비그램', '육아그램', '신혼', '신혼부부', '닭띠맘', '원숭이띠아기', 
          '가족스타그램', '닭띠아기', '딸스타그램']
animal = ['고양이', '반려견', '강아지', '냥스타그램', '개스타그램', '펫스타그램', '댕댕이', '캣스타그램',
             '독스타그램', '견스타그램', '푸들', '말티즈', '반려묘', '멍멍이', '포메라니안', '반려동물']
health = ['운동', '운동하는여자', '헬스타그램', '헬스', '운동하는남자', '운동스타그램']
mountain = [ '여행스타그램',  '산스타그램', '산', '여행에미치다', '휴가', '풍경','등산', '등산스타그램',  '등산그램' ]
ocean = ['여행스타그램', '바다스타그램', '바다', '여행에미치다', '휴가', '풍경' , '광안리', '해운대', '바다여행', '바다뷰' ]

total_keywords = flower + food + drink + fashion + nailart + love + baby + animal + health + mountain + ocean
key_counts = {}
for words in total_keywords:
    counts = 0
    for tags in img_df["tags"].values:
        if words in tags:
            counts += 1
    key_counts[words] = str(counts)

In [4]:
# Function to encode a given image into a vector of size (2048, )
def encode(image_path):
    global model_new
    img = keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    # Convert image to numpy array of 3-dimensions
    x = img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    fea_vec = model_new.predict(x) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec   # 이 데이터가 input data로 사용됨

In [5]:
def image_encode( keyword, img_lst = []):
    global object_model, scenery_model
    image_path='./static/test_img/'
    data=[]
    for image in tqdm(img_lst):
        path=image_path+image
        data.append( np.array(encode(path)) )
    data = np.array(data)
    if keyword in health or keyword in mountain or keyword in ocean:
        # 풍경검출 결과
        y_scenery_pred=[]
        label=scenery_model.predict(data)
        for sample in label:
            y_scenery_pred.append([1 if i>=0.5 else 0 for i in sample ] )
        return np.array(y_scenery_pred)
    else:
        # 객체검출 결과
        y_object_pred=[]
        label=object_model.predict(data)
        for sample in label:
            y_object_pred.append([1 if i>=0.5 else 0 for i in sample ] )
        return np.array(y_object_pred)

In [6]:
def keyword_img_match(keyword):
    global img_df
    img_lst = []
    for i in img_df.index:
        if keyword in img_df.iloc[i,1]:
            img_lst.append(img_df.iloc[i,0])
    return img_lst

In [7]:
def keyword_search_len(keyword):
    lst = 0
    for i in img_df.index:
        if keyword in img_df.iloc[i,1]:
            lst+=1
    return lst

In [8]:
def tag_detecting(keyword, predict_list, img_lst):
    filtered_list = []

    #  객체검출
    if keyword in flower:
        for i in range(len(predict_list)):
            if predict_list[i][6] == 1:
                filtered_list.append(img_lst[i])
        return filtered_list
        
    elif keyword in food :
        for i in range(len(predict_list)):
            if predict_list[i][0] == 1:
                filtered_list.append(img_lst[i])
        return filtered_list
        
    elif keyword in drink :
        for i in range(len(predict_list)):
            if predict_list[i][1] == 1:
                filtered_list.append(img_lst[i])
        return filtered_list
        
    elif keyword in fashion :
        for i in range(len(predict_list)):
            if predict_list[i][2] == 1 or predict_list[i][3] == 1:
                filtered_list.append(img_lst[i])
        return filtered_list
        
    elif keyword in nailart :
        for i in range(len(predict_list)):
            if predict_list[i][5] == 1:
                filtered_list.append(img_lst[i])
        return filtered_list
        
    elif keyword in love :
        for i in range(len(predict_list)):
            if predict_list[i][2] == 1 and predict_list[i][3] == 0:
                filtered_list.append(img_lst[i])
        return filtered_list
        
    elif keyword in baby :
        for i in range(len(predict_list)):
            if predict_list[i][3] == 1:
                filtered_list.append(img_lst[i])
        return filtered_list
        
    elif keyword in animal :
        for i in range(len(predict_list)):
            if predict_list[i][4] == 1:
                filtered_list.append(img_lst[i])
        return filtered_list
        
    
    #  풍경검출
    elif keyword in health :
        for i in range(len(predict_list)):
            if predict_list[i][2] == 1:
                filtered_list.append(img_lst[i])
        return filtered_list
        
    elif keyword in mountain :
        for i in range(len(predict_list)):
            if predict_list[i][0] == 1:
                filtered_list.append(img_lst[i])
        return filtered_list
        
    elif keyword in ocean :
        for i in range(len(predict_list)):
            if predict_list[i][1] == 1:
                filtered_list.append(img_lst[i])
        return filtered_list
        

### 필터링 되는 코드

In [91]:
app = Flask(__name__)

filename = ""
img_lst = []
num = 12

# 웹 페이지 접속시 메인 화면
@app.route('/',  methods=['GET'])
def main_loading():
    return render_template("index.html")

# 키워드 표출(비동기통신)
@app.route("/keywords", methods = ["POST"])
def keywords_search():
    global total_keywords, key_counts
    keyword = request.form["keyword"]
    
    keywords = {}
    cnt = 0
    # 검색어가 있는 키워드 필터링
    for word in total_keywords:
        if keyword in word:
                                  # 키워드  ,    갯수
            keywords[str(cnt)] = [word, key_counts[word]]
            cnt+=1
    return json.dumps(keywords)

# 검색 페이지
@app.route("/search", methods=["GET"])
def search_page():
    global img_lst, num
    num = 12
    keyword = request.args.get("keyword")
    status = request.args.get("status")
    img_lst = keyword_img_match(keyword)
    if status == "filtering":
        status = "original"
        return render_template("search.html",img_names = img_lst[:num],
                                keyword = keyword, img_len = len(img_lst),
                                mtime = datetime.datetime.now().time().strftime("%d%H%M%S"),
                                status = status)
    elif status == "original":
        status = "filtering"
        labeld_lst = image_encode(keyword, img_lst[:num])
        filtered_lst = tag_detecting(keyword, labeld_lst, img_lst[:num])
        if len(img_lst) > 12:
            while len(filtered_lst) != 12:
                if num > len(img_lst): break
                labeld_lst = image_encode(keyword, [img_lst[num]])
                filtered_lst = filtered_lst + tag_detecting(keyword, labeld_lst, [img_lst[num]])
                num += 1
        return render_template("search.html",img_names = filtered_lst,
                                keyword = keyword, img_len = len(img_lst),
                                mtime = datetime.datetime.now().time().strftime("%d%H%M%S"),
                                status = status)

# 검색페이지에서 스크롤시 새로고침
@app.route("/image_append", methods=["POST"])
def image_append():
    global num, img_lst
    keyword = request.get_json()["keyword"]
    status = request.get_json()["status"]
    # num = int(request.get_json()["num"])
    if num == len(img_lst):
        return json.dumps({})
    start_num = num
    num += 12
    if num > len(img_lst):
        num = len(img_lst)
    if status == "original":
        img_dict = [{names:names} for names in img_lst[start_num:num]]
        return json.dumps(img_dict)
    elif status == "filtering":
        labeld_lst = image_encode(keyword, img_lst[start_num:num])
        filtered_lst = tag_detecting(keyword, labeld_lst, img_lst[start_num:num])
        while len(filtered_lst) != 12:
            if num == len(img_lst): break
            labeld_lst = image_encode(keyword, [img_lst[num]])            
            filtered_lst = filtered_lst + tag_detecting(keyword, labeld_lst, [img_lst[num]])
            num += 1
        img_dict = [{names:names} for names in filtered_lst]
        return json.dumps(img_dict)

  
if __name__ == "__main__":

    app.run()  

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [25/Jul/2022 16:33:10] "GET /search?keyword=아침&status=filtering HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2022 16:33:10] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:33:10] "GET /static/img/insta_logo.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2022 16:33:10] "GET /static/img/home_icon.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2022 16:33:10] "GET /static/img/plus_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:33:10] "GET /static/nav.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2022 16:33:10] "GET /static/img/heart.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:33:10] "GET /static/search_script.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2022 16:33:10] "GET /static/img/cancle_button.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2022 16:33:10] "GET /static/img/paper_plane.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2022 16:33:10] "GET /static/img/compass_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul

start12
24
[{'9003868.jpg': '9003868.jpg'}, {'9002146.jpg': '9002146.jpg'}, {'9002142.jpg': '9002142.jpg'}, {'9007084.jpg': '9007084.jpg'}, {'9007393.jpg': '9007393.jpg'}, {'9002409.jpg': '9002409.jpg'}, {'9022183.jpg': '9022183.jpg'}, {'9002172.jpg': '9002172.jpg'}, {'9011065.jpg': '9011065.jpg'}, {'9014391.jpg': '9014391.jpg'}, {'9014998.jpg': '9014998.jpg'}, {'9002383.jpg': '9002383.jpg'}]


127.0.0.1 - - [25/Jul/2022 16:33:13] "POST /image_append HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2022 16:33:13] "GET /static/test_img/9003385.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:33:13] "GET /static/test_img/9002171.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:33:13] "GET /static/test_img/9023740.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:33:13] "GET /static/test_img/9018215.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:33:13] "GET /static/test_img/9002402.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:33:13] "GET /static/test_img/9002119.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:33:13] "GET /static/test_img/9014943.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:33:13] "GET /static/test_img/9019379.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:33:13] "GET /static/test_img/9002141.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:33:13] "GET /static/test_img/9002127.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:33:13] "GET /static/test_im

start24
36
[{'9003385.jpg': '9003385.jpg'}, {'9002171.jpg': '9002171.jpg'}, {'9023740.jpg': '9023740.jpg'}, {'9018215.jpg': '9018215.jpg'}, {'9002402.jpg': '9002402.jpg'}, {'9002119.jpg': '9002119.jpg'}, {'9014943.jpg': '9014943.jpg'}, {'9019379.jpg': '9019379.jpg'}, {'9002141.jpg': '9002141.jpg'}, {'9002127.jpg': '9002127.jpg'}, {'9014050.jpg': '9014050.jpg'}, {'9002283.jpg': '9002283.jpg'}]


...

100%|██████████| 1/1 [00:00<00:00,  2.61it/s]
127.0.0.1 - - [25/Jul/2022 16:34:52] "GET /search?keyword=아침&status=original HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2022 16:34:52] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:34:53] "GET /static/img/insta_logo.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:34:53] "GET /static/img/home_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:34:53] "GET /static/test_img/9002603.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:34:53] "GET /static/img/profile_img.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:34:53] "GET /static/img/paper_plane.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:34:53] "GET /static/nav.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:34:53] "GET /static/search_script.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:34:53] "GET /static/test_img/9002263.jpg?q=01163452 HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2022 16:34:53] "GET /static/test_img/9003572.jpg?q=01163452 HTTP/1.1" 200 -
127.0

start17


100%|██████████| 1/1 [00:00<00:00,  3.43it/s]
127.0.0.1 - - [25/Jul/2022 16:35:06] "POST /image_append HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2022 16:35:06] "GET /static/test_img/9002409.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:35:06] "GET /static/test_img/9002127.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:35:06] "GET /static/test_img/9002172.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:35:06] "GET /static/test_img/9014998.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:35:06] "GET /static/test_img/9002171.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:35:06] "GET /static/test_img/9003385.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:35:06] "GET /static/test_img/9014050.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:35:08] "POST /image_append HTTP/1.1" 200 -


start36


127.0.0.1 - - [25/Jul/2022 16:36:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2022 16:36:21] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:36:21] "GET /static/img/insta_logo.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:36:21] "GET /static/img/back_button.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:36:21] "GET /static/img/home_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:36:21] "GET /static/img/story_border.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:36:21] "GET /static/img/profile_img.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:36:21] "GET /static/img/next_button.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:36:21] "GET /static/img/story_image.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:36:21] "GET /static/img/dots.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:36:21] "GET /static/img/content_img.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 16:36:21] "GET /static/img/paper_plane.png HTTP/1.1" 304 -
127.0.0.1 - -

### 필터링 안되는 코드 (비교 예시)

In [12]:
app = Flask(__name__)

filename = ""
img_lst = []
num = 12

# 웹 페이지 접속시 메인 화면
@app.route('/',  methods=['GET'])
def main_loading():
    return render_template("index.html")

# 키워드 표출
@app.route("/keywords", methods = ["POST"])
def keywords_search():
    global total_keywords, key_counts
    keyword = request.form["keyword"]
    
    keywords = {}
    cnt = 0
    # 검색어가 있는 키워드 필터링
    for word in total_keywords:
        if keyword in word:
                                  # 키워드  ,    갯수
            keywords[str(cnt)] = [word, key_counts[word]]
            cnt+=1
    return json.dumps(keywords)

# 검색 페이지
@app.route("/search", methods=["GET"])
def search_page():
    global img_lst, labeld_lst, num
    num = 12
    keyword = request.args.get("keyword")
    img_lst = keyword_img_match(keyword)
    # labeld_lst = image_encode(keyword, img_lst[:num])
    # filtered_lst = tag_detecting(keyword, labeld_lst, img_lst[:num])
    # if len(img_lst) > 12:
    #     while len(filtered_lst) > 12:
    #         if num > len(img_lst): break
    #         labeld_lst = image_encode(keyword, img_lst[num])
    #         filtered_lst = filtered_lst + tag_detecting(keyword, [labeld_lst], [img_lst[num]])
    #         num += 1

    return render_template("search.html",img_names = img_lst[:12], keyword = keyword, img_len = len(img_lst))

# 검색페이지에서 스크롤시 새로고침
@app.route("/image_append", methods=["POST"])
def image_append():
    global num
    keyword = request.get_json()["keyword"]
    # num = int(request.get_json()["num"])
    start_num = num
    num += 12

    if num < len(img_lst):
        # labeld_lst = image_encode(keyword, img_lst[start_num:num])
        # filtered_lst = tag_detecting(keyword, labeld_lst, img_lst[start_num:num])
        # while len(filtered_lst) > 12:
        #     if num > len(img_lst): break
        #     labeld_lst = image_encode(keyword, img_lst[num])            
        #     filtered_lst = filtered_lst + tag_detecting(keyword, [labeld_lst], [img_lst[num]])
        #     num += 1
        # img_dict = [{names:names} for names in filtered_lst]

        # 예시를 위한 코드
        img_dict = [{names:names} for names in img_lst[start_num:num]]
        #####################
        
        return json.dumps(img_dict)

    else:
        img_dict = [{names:names} for names in img_lst[start_num:]]
        return json.dumps(img_dict)

  
if __name__ == "__main__":

    app.run()  

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
100%|██████████| 1/1 [00:00<00:00,  2.05it/s]
127.0.0.1 - - [25/Jul/2022 10:25:23] "GET /search?keyword=아침 HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2022 10:25:23] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 10:25:23] "GET /static/img/insta_logo.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 10:25:23] "GET /static/nav.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 10:25:23] "GET /static/search_script.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 10:25:23] "GET /static/img/cancle_button.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 10:25:23] "GET /static/img/home_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 10:25:23] "GET /static/img/paper_plane.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 10:25:23] "GET /static/img/plus_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 10:25:23] "GET /static/img/compass_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2022 10:25:24] "GET /static/nav.js HTTP/1.1" 304 -